In [73]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import re
import nltk
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import spacy
from heapq import nlargest
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
from collections import defaultdict





# Read the text file
with open('article.txt', 'r') as file:
    text = file.read()

# Split the text into sentences
sentences = sent_tokenize(text)

# Store the sentences in a DataFrame
df = pd.DataFrame(sentences, columns=['Sentence'])

# Save the DataFrame to a CSV file
df.to_csv('sentences.csv', index=False)

# Optionally, print the first few rows of the DataFrame
print(df.head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mahim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                            Sentence
0  Telegram chief executive Pavel Durov has been ...
1  Mr Durov was detained after his private jet ha...
2  According to officials the 39-year-old billion...
3  The investigation is reportedly about a lack o...
4  The app is accused of failure to cooperate wit...


In [74]:
df = pd.read_csv("sentences.csv")

In [75]:
df

,Sentence
0,Telegram chief executive Pavel Durov has been ...
1,Mr Durov was detained after his private jet ha...
2,According to officials the 39-year-old billion...
3,The investigation is reportedly about a lack o...
4,The app is accused of failure to cooperate wit...
5,Telegram has previously denied having insuffic...
6,Pavel Durov was born in Russia and now lives i...
7,He holds dual citizenship of the United Arab E...
8,"Telegram is particularly popular in Russia, Uk..."
9,"The app was banned in Russia in 2018, after a ..."


In [76]:

stop_words = set(stopwords.words('english'))



In [77]:
with open('article.txt', 'r') as file :
    text = file.read()

In [78]:
print(len(text))
text

3121


'Telegram chief executive Pavel Durov has been arrested by French police at an airport north of Paris.\nMr Durov was detained after his private jet had landed at Le Bourget Airport, French media reported.\nAccording to officials the 39-year-old billionaire was arrested under a warrant for offences related to the popular messaging app. The investigation is reportedly about a lack of moderation, with Mr Durov accused of failing to take steps to curb criminal uses of Telegram.\nThe app is accused of failure to cooperate with law enforcement over drug trafficking, child sexual content and fraud. Telegram has previously denied having insufficient moderation.\nPavel Durov was born in Russia and now lives in Dubai, where Telegram is based. He holds dual citizenship of the United Arab Emirates and France.\nTelegram is particularly popular in Russia, Ukraine and former Soviet Union states.\nThe app was banned in Russia in 2018, after a previous refusal by him to hand over user data. The ban was

In [95]:

sen = text.split(' ')
size = int(len(sen) * 0.2)
size1 = int(len(sen))
priority = []
for i in range(0, size):
    priority.append(sen[i])
    priority.append(sen[size1 - i -1])


    

In [113]:

# Ensure you have the stopwords downloaded from NLTK
STOP_WORDS = set(stopwords.words('english'))

def textSummarizer(text, percentage):
    # Load the model into spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Pass the text into the nlp function
    doc = nlp(text)
    
    # Frequency table for words
    freq_of_word = defaultdict(int)
    for token in doc:
        
        # Consider only nouns and significant words
        if token.text.lower() not in STOP_WORDS and token.text not in punctuation:
            
            if token.pos_ == 'NOUN':
                
                # Give more weight to nouns (e.g., multiply by 2)
                freq_of_word[token.text] += 2
            if token.text in priority:
                freq_of_word[token.text] += 4
            else:
                freq_of_word[token.text] += 1
    
    # Maximum frequency of word
    max_freq = max(freq_of_word.values(), default=1)  # Avoid division by zero
    
    # Normalization of word frequency
    for word in freq_of_word.keys():
        freq_of_word[word] = freq_of_word[word] / max_freq
        
    # Score sentences
    sent_scores = defaultdict(int)
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() in freq_of_word:
                sent_scores[sent] += freq_of_word[word.text.lower()]
    
    # Determine number of sentences for summary
    len_tokens = int(len(list(doc.sents)) * percentage)
    
    # Select sentences with the highest scores
    summary = nlargest(n=len_tokens, iterable=sent_scores, key=sent_scores.get)
    
    # Convert doc.sents to a list for indexing
    sent_list = list(doc.sents)
    
    # Prepare final summary in original order
    final_summary = sorted(summary, key=lambda x: sent_list.index(x))
    
    # Convert to a string
    summary_text = " ".join([sent.text for sent in final_summary])
    
    return summary_text


In [115]:
print("The number of words in given text is :")
final_summary = textSummarizer(text, 0.4)
print(len(final_summary))
print("The summary is:")
print(final_summary)

The number of words in given text is :
1404
The summary is:
Mr Durov was detained after his private jet had landed at Le Bourget Airport, French media reported.
 According to officials the 39-year-old billionaire was arrested under a warrant for offences related to the popular messaging app. The investigation is reportedly about a lack of moderation, with Mr Durov accused of failing to take steps to curb criminal uses of Telegram.
 The app is accused of failure to cooperate with law enforcement over drug trafficking, child sexual content and fraud. He left Russia in 2014 after refusing to comply with government demands to shut down opposition communities on his VKontakte social media platform, which he sold.
 X owner Elon Musk, who has faced extensive criticism over moderation and material hosted by his own social media site, posted repeatedly about the situation.
 "
Telegram allows groups of up to 200,000 members, which critics have argued makes it easier for misinformation to spread,